In [3]:
from glob import glob
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from astropy import stats
import pandas as pd
import csv
import os
from dl import queryClient as qc

C:\Users\kylem\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\kylem\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.JPIJNSWNNAN3CE6LLI5FWSPHUT2VXMTH.gfortran-win_amd64.dll
C:\Users\kylem\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.WCDJNK7YVMPZQ2ME2ZZHJJRJ3JIKNDB7.gfortran-win_amd64.dll
  stacklevel=1)


In [4]:
if not os.path.exists('results'):
    os.makedirs('results')
outdir = 'results/plots'
if not os.path.exists(outdir):
    os.makedirs(outdir)

In [5]:
os.getcwd()

'D:\\msu\\DavidNidever\\TempletFit'

In [6]:
interesting = np.loadtxt('goldsample\golden_original.txt',dtype=str)

In [7]:
tmps = np.loadtxt('templets/LaydenTemplates.txt',delimiter=',')
tmps.shape

(51, 8)

In [8]:
def get_data(objname):
    """Query the object by name, extract light curves, estimated period and filters."""
    df=qc.query(sql="""SELECT meas.* 
                     FROM nsc_dr2.meas
                     WHERE objectid='{:s}'""".format(objname),
              fmt='pandas',
              profile='db01')
    # Always uses this filter order
    forder = ['u','g','r','i','z']
    crv=[]
    best_periods = []
    fltrs=[]
    for f in forder:
        selfltr = (df['filter'] == f)
        selfwhm = (df['fwhm'] <= 4.0)
        sel = selfltr & selfwhm
        t = df['mjd'][sel].values
        y = df['mag_auto'][sel].values
        dy = df['magerr_auto'][sel].values
        if len(t) < 25:
            continue
        
        # get estimated period
        pout = get_ls_period(t,y,objname=objname+'_'+f,outdir=outdir)
        best_periods.append(pout)
        # save light curve
        crvi = np.vstack((t,y,dy)).T
        crv.append(crvi[np.argsort(crvi[:,0])])
        # record which filter this applies to
        fltrs.append(f)
    
    # Find mean period
    period = np.mean(best_periods)
    return crv, period, fltrs

def get_ls_period(t,y,min_freq=1./1.,max_freq=1./0.1,objname='_',outdir='results'):
    """Use Lomb-Scargle periodogram to get an estimate on period"""
    
    ls = stats.LombScargle(t, y)
    frequency, power = ls.autopower(minimum_frequency=min_freq,maximum_frequency=max_freq)
    period = 1./frequency # period is the inverse of frequency
    
    best_period = period[np.argmax(power)]
    
    plot_periodogram(period,power,best_period,objname=objname,outdir=outdir)
    return best_period

def plot_periodogram(period,power,best_period=None,objname='',ax=None,outdir='results'):
   
    if ax is None:
        fig, ax = plt.subplots(figsize=(10,7))
        
    ax.plot(period,power,lw=0.1)
    ax.set_xlabel('period (days)')
    ax.set_ylabel('relative power')
    ax.set_title(objname)
    
    if best_period is not None:
        ax.axvline(best_period,color='r');
        ax.text(0.03,0.93,'period = {:.3f} days'.format(best_period),transform=ax.transAxes,color='r')
    fig.savefig(outdir+'/{}_periodogram.png'.format(objname))
    plt.close(fig)
    return

def double_tmps(tmps):
    tmps2=[]
    for f in range(len(tmps)):
        tmps2.append(np.tile(tmps[f],(2,1)))
        tmps2[f][:,int(len(tmps2[f][0])/2):,0] += 1
    return tmps2

def update_pinit(pars,period):
    pinit = ()
    for i in range(len(pars)):
        pinit += (tuple(pars[i,:-1]),)
    pinit += (period,)
    return pinit

def RemoveOutliers(crv,tmps,pars,period):
    n = pars[:,-1].astype(int)
    crv_in = []
    for i in range(len(crv)):
        f = interp1d(tmps[i][n[i],:,0],tmps[i][n[i],:,1]*pars[i,1]+pars[i,2])
        phase = (crv[i][:,0]/period-pars[i,0]) %1
        dif = abs(crv[i][:,1]-f(phase))
        crv_in.append(crv[i][dif<utils.mad(dif)*5])
    return crv_in

def double_period(crv,pars,period):
    crv2 = []
    for i in range(len(crv)):
        crv2.append(crv[i].copy())
        crv2[i][:,1] -= pars[i,2]
        
        crv2[i][:,0] = (crv2[i][:,0]/period-pars[i,0])%1
        crv2[i] = np.tile(crv2[i].T,2).T
        crv2[i][int(len(crv2[i])/2):,0] += 1
        crv2[i] = crv2[i][crv2[i][:,0].argsort()]
        
    return crv2

In [9]:
class tmpfitter:
    def __init__ (self, tmps):
        self.fltr=0
        self.n=0
        self.tmps=tmps

    def model(self, t, t0, amplitude, yoffset):
        # modify the template using peak-to-peak amplitude, yoffset
        # fold input times t by period, phase shift to match template
        xtemp = self.tmps[:,0]
        ytemp = self.tmps[:,self.n]*amplitude + yoffset
        ph = (t - t0) %1
        #print((ph[0],period,t0%1))
        #print((period,t0,amplitude,yoffset))
        # interpolate the modified template to the phase we want
        return interp1d(xtemp,ytemp)(ph)


def tmpfit(crv,tmps,p,init,w=.1,steps=21,n=1):
    fitter = tmpfitter(tmps)
    
    # Search periods from center outward
    lsteps = int(steps/2+.5)
    rsteps = steps - lsteps
    pl = np.linspace(p-w,p,lsteps)
    pr = np.linspace(p+w,p,rsteps,endpoint=False)
    plist = np.zeros(pl.size+pr.size)
    plist[0::2] = np.flip(pl)
    plist[1::2] = np.flip(pr)
    plist = plist[plist>0]
    
    pars = np.zeros((len(tmps),4))
    minsumx2 = 10**50
    minp = 0
    for p in plist:
        sumx2=0
        ppars=np.zeros((len(tmps),4))
        for f in range(len(tmps)):
            fitter.fltr = f
            phase = crv[f][:,0]/p%n #1 for one period, 2 for two periods
            minx2 = 10**50
            for i in range(len(tmps[f])):
                fitter.n = i+1
                try:
                    tpars, cov = curve_fit(fitter.model, phase, crv[f][:,1], 
                                          bounds = ((-.5,0,-50),(.5,20,50)),
                                          sigma=crv[f][:,2], p0=init[f], maxfev=500)
                except RuntimeError:
                    continue
                
                x2 = sum((fitter.model(phase,tpars[0],tpars[1],tpars[2])-crv[f][:,1])**2/crv[f][:,2]**2)
                if x2 < minx2:
                    ppars[f,:-1] = tpars
                    ppars[f,-1] = i
                    minx2 = x2
            
            sumx2 += minx2
            if sumx2 > minsumx2:
                break
        if sumx2 < minsumx2:
            minsumx2 = sumx2
            minp = p
            pars = ppars
    npoints=0
    for i in range(len(crv)):
        npoints += len(crv[i])
    return pars, minp, minsumx2/npoints

In [10]:
c,p,f = get_data(interesting[1])

In [11]:
def fit_plot(objname,file):
    crv,p,fltrs = get_data(objname)
    if len(fltrs) == 0:
        return
    
    init = ()
    for ltcrv in crv:
        init += ((0.0,max(ltcrv[:,1])-min(ltcrv[:,1]),0.0),)
    
    pars, p, x2 = tmpfit(crv,tmps,p,init,w=.1,steps=25)
    crv_in = RemoveOutliers(crv,tmps,pars,p)
    pinit = update_pinit(pars,p)
    pars_in,p_in,x2 = tmpfit(crv_in,tmps, pinit,w=.01,steps=25)
    
    crv2 = double_period(crv,pars_in,p_in)
    tmps2= double_tmps(tmps)
    n = pars[:,-1].astype(int)
    
    colors = []
    for f in fltrs:
        if f == 'r' or f == 'g':
            colors.append(f)
        else:
            colors.append('black')

    #Check if each filter is consistent with RR type (RRab or RRc)
    consistent = True
    for i in range(len(typs)):
        for j in range(i+1,len(typs)):
            if typs[i][n[i]] != typs[j][n[j]]:
                consistent = False
                break
        if not consistent:
            break
    if consistent:
        typ = typs[0][n[0]]
    else:
        typ = '???'
    fig, ax = plt.subplots(len(fltrs), figsize=(10,7.5), sharex=True, sharey=True)
    if len(fltrs) == 1:
        ax = [ax]
    for i in range(len(fltrs)):
        crvmean = mean(crv2[i][:,1])
        ax[i].scatter(crv2[i][:,0],crv2[i][:,1]-crvmean,c=colors[i])
        ax[i].plot(tmps2[i][n[i],:,0],tmps2[i][n[i],:,1]*pars_in[i,1]-crvmean,c='black')
        ax[i].invert_yaxis()
        ax[i].set_ylabel(fltrs[i], fontsize=18)

    ax[-1].set_xlabel('Phase', fontsize=16)
    ax[0].set_title("Object: {}    Period: {:.3f} d    Type: {}".format(objname,p_in,typ), fontsize=20)
    fig.savefig('results/plots/{}.pdf'.format(objname))
    
    file.write("{},{:.3f},{:.3f},\n".format(objname,x2,p_in))
    for i in range(len(fltrs)):
        file.write("{:.3f},{:.3f},{:.3f},{}\n".format(pars_in[i][0],pars_in[i][1]/2,pars_in[i][2],tmpnames[i][n[i]][9:]))
    plt.close(fig)

In [13]:
gldorig = np.loadtxt('goldsample\golden_original.txt',delimiter=',',dtype=str)
gldrrab = np.loadtxt('goldsample\golden_RRab.txt',delimiter=',',dtype=str)

In [17]:
np.union1d(gldorig,gldrrab)

array(['100047_2267', '100413_6560', '100555_10583', ..., '99770_15284',
       '99796_35', '99850_3275'], dtype='<U13')

In [83]:
from astropy.table import Table
ampt=Table([gldrrab],names=['id'])
ampt['uamp'] = -99.99
ampt['gamp'] = -99.99
ampt['ramp'] = -99.99
ampt['iamp'] = -99.99
ampt['zamp'] = -99.99

In [84]:
import csv
fdict = {'u':1,'g':2,'r':3,'i':4,'z':5}
for i,name in enumerate(gldrrab):
    with open('results\{}_parameters.csv'.format(name), newline='') as csvfile:
        b,c,f = get_data(name)
        reader = csv.reader(csvfile, delimiter=',')
        first = True
        amps = []
        for row in reader:
            if first:
                first = False
                continue
            if row == ['---']:
                continue
            amps.append(float(row[1]))
        for j in range(len(f)):
            n = fdict[f[j]]
            ampt[i][n] = amps[j]

In [86]:
ampt.write("amp_rrab.fits",format='fits')